In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import tokenization
import collections
import ast
import time
from nltk.tokenize import TweetTokenizer

In [2]:
columns = ['character1','character2','movietitle','conversation']

movie_conver = []
with open('data/movie_conversations.txt','r') as file:
    for l in file:
        lines = l.split('+++$+++')
        line_content =  eval(lines[-1].rstrip())
        lines = [x.strip() for x in lines[:-1]]
        lines.append(line_content)
        movie_conver.append(lines)
movie_conver = pd.DataFrame(movie_conver, columns=columns)

In [3]:
columns = ['line','character_id','movietitle','character_name','text']

movie_lines = []
with open('data/movie_lines.txt','r', errors='ignore') as file:
    for l in file:
        lines = l.split('+++$+++')
        lines[-1] = lines[-1].rstrip()
        lines = [x.strip() for x in lines]
        movie_lines.append(lines)
movie_lines = pd.DataFrame(movie_lines, columns=columns)

In [4]:
assert len(movie_lines['line'].unique()) == len(movie_lines['line'])
movie_lines.set_index('line',inplace=True)

In [14]:
movie_title = movie_conver['movietitle'].unique()
select_title = movie_title[:300]

select_movie_conver = movie_conver[np.isin(movie_conver['movietitle'], select_title)]
select_movie_lines = movie_lines[np.isin(movie_lines['movietitle'], select_title)]

In [15]:
class Example():
    
    def __init__(self,input_sentences, output_sentences):
        
        self.input_sentences = input_sentences
        self.output_sentences = output_sentences
               

In [16]:
def convert_toIndex(tokens, word_dict):

    index_features = []
    for token in tokens:
        if not token in word_dict:
            word_dict[token] = len(word_dict)

        index_features.append(word_dict[token])
    return index_features

In [17]:
def truncate_sequence(text_a, text_b, max_len):
    
    while len(text_a) + len(text_b) + 3 >= max_len:
        if len(text_b) > len(text_a):
            text_b.pop()
        else:
            text_a.pop()

In [18]:
def convert_toExample(num_i, conversation, tokenizer, max_input_seq, max_output_seq, word_dict):
#     convert text into examples
    
    num_convers = len(conversation)
    for i, _ in enumerate(conversation):
        
        if i < num_convers - 1:
            num_i += 1
            
            input_texts = tokenizer.tokenize(conversation[i])
            response = tokenizer.tokenize(conversation[i+1])
#             response = tokenizer.tokenize(conversation[i+2])          
#             truncate_sequence(text_q, text_p, max_input_seq)              
#             input_texts = text_q + ['[SEQ]'] + text_p
            if len(input_texts) == 0 or len(response) == 0:
                continue
            
            if len (input_texts) > max_input_seq or len (response) > max_output_seq - 1:
                continue

#             if len (input_texts) > max_input_seq:
#                 input_texts = input_texts[:max_output_seq]
            
#             if len (response) > max_output_seq - 1:
#                 response = response[:max_output_seq - 1]
                
            response = response + ['[END]']
        
            input_id = convert_toIndex(input_texts, word_dict)
            output_id = convert_toIndex(response, word_dict)
            
            if len(input_id) < max_input_seq:
                input_id.extend([0] * (max_input_seq - len(input_id)))
                
            if len(output_id) < max_output_seq:
                output_id.extend([0] * (max_output_seq - len(output_id)))
                
            assert len(input_id) == max_input_seq
            assert len(output_id) == max_output_seq
            
            if num_i % 5000 == 0:
                tf.logging.info('input text: %s' % ' '.join(input_texts))
                tf.logging.info('input id: %s' % ' '.join([str(x) for x in input_id]))
                tf.logging.info('output text: %s' % ' '.join(response))
                tf.logging.info('output id: %s' % ' '.join([ str(x) for x in output_id]))
            
            yield num_i, Example(input_id, output_id)
        

In [19]:
def build_dataSet(movie_conver, movie_lines, max_input_seq, max_output_seq, tokenizer):
    
    word_dict = collections.OrderedDict()
    word_dict['#UNTOKEN#'] = 0
    word_dict['[START]'] = 1
    num_i = 0
    
    examples = []
    for index, row in movie_conver.iterrows():
        line_index = row['conversation']
        conversation = movie_lines.loc[line_index,'text']
        
        if len(conversation) > 2:
            for num_i, example in convert_toExample(num_i, conversation, tokenizer, max_input_seq, max_output_seq, word_dict):
                examples.append(example)
    return word_dict, examples
    

In [20]:
def save_data(examples, file_path):
    writer = tf.python_io.TFRecordWriter(file_path)
    
    for example in examples:
        features = collections.OrderedDict()
        features["input_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=example.input_sentences))
        features["output_ids"] = tf.train.Feature(int64_list=tf.train.Int64List(value=example.output_sentences))

        tf_example = tf.train.Example(features=tf.train.Features(feature=features))
        writer.write(tf_example.SerializeToString())
    writer.close()
    

In [15]:
# tokenizer = tokenization.FullTokenizer('data/vocab.txt')
tokenizer = TweetTokenizer()

In [22]:
max_input_seq = 10
max_output_seq = 10

word_dict, examples = build_dataSet(select_movie_conver, select_movie_lines, max_input_seq, max_output_seq, tokenizer)
print(len(examples))

INFO:tensorflow:input text: Thank-Q !
INFO:tensorflow:input id: 3489 114 0 0 0 0 0 0 0 0
INFO:tensorflow:output text: I thought only hookers drank those things . [END]
INFO:tensorflow:output id: 35 111 317 3522 3523 1985 1408 9 12 0
INFO:tensorflow:input text: Where ... do those men come from , George ?
INFO:tensorflow:input id: 62 16 31 1985 554 498 147 14 1653 30
INFO:tensorflow:output text: Africa , of course . [END]
INFO:tensorflow:output id: 4357 14 49 1056 9 12 0 0 0 0
INFO:tensorflow:input text: I'll be out in a minute .
INFO:tensorflow:input id: 238 346 164 159 6 129 9 0 0 0
INFO:tensorflow:output text: I'm sorry about my parents . [END]
INFO:tensorflow:output id: 83 604 60 47 1241 9 12 0 0 0
INFO:tensorflow:input text: And the baby ?
INFO:tensorflow:input id: 121 39 1337 30 0 0 0 0 0 0
INFO:tensorflow:output text: He's a big one . [END]
INFO:tensorflow:output id: 607 6 2173 8 9 12 0 0 0 0
INFO:tensorflow:input text: What's that , Dom ? What's the bright side ?
INFO:tensorflow:

In [23]:
from sklearn.model_selection import train_test_split
train_example, val_examples = train_test_split(examples, test_size = 0.2, random_state = 3)
save_data(train_example, 'data/train')
save_data(val_examples, 'data/val')

In [24]:
del movie_conver, select_movie_conver
del movie_lines, select_movie_lines
del train_example, val_examples, examples

In [25]:
import json
with open('data/vocab.json', 'w') as fp:
    json.dump(word_dict, fp)

In [15]:
def encoder(input_ids, embeding, vocab_size, embeding_size):
    
    with tf.variable_scope('encoder', reuse = tf.AUTO_REUSE):
    
        input_embeding = tf.nn.embedding_lookup(embeding, input_ids)

        rnn_layers = [tf.nn.rnn_cell.GRUCell(size) for size in [512,256]]

        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)

        sequence_length = tf.reduce_sum(tf.cast(input_ids > 0, dtype = tf.int32), axis=-1)

        outputs, state = tf.nn.dynamic_rnn(multi_rnn_cell, input_embeding, dtype=tf.float32, sequence_length = sequence_length)
    
    return outputs, state, sequence_length
    

In [16]:
def decoder(output_ids, embeding, vocab_size, attention_define, padding_size = 10, training = True):

    with tf.variable_scope('decoder', reuse = tf.AUTO_REUSE):
        
        batch = tf.shape(output_ids)[0]
        
        rnn_layers = [tf.nn.rnn_cell.GRUCell(size) for size in [512,256]]

        multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
        
        encoder_output, encoder_state, encoder_length, attention_unit = attention_define
        
        attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(attention_unit, encoder_output, encoder_length)

        attention_cell = tf.contrib.seq2seq.AttentionWrapper(multi_rnn_cell, attention_mechanism, attention_layer_size = None)

        attention_cell = tf.contrib.rnn.OutputProjectionWrapper(attention_cell, vocab_size, reuse=tf.AUTO_REUSE)

        initial_state = attention_cell.zero_state(dtype = tf.float32, batch_size = batch)
        initial_state = initial_state.clone(cell_state=encoder_state)
        
        if training:          
            input_embeding = tf.nn.embedding_lookup(embeding, output_ids)
            sequence_length = tf.count_nonzero(output_ids, axis = -1, dtype = tf.int32)
            helper = tf.contrib.seq2seq.TrainingHelper(
                        inputs = input_embeding,
                        sequence_length = sequence_length,
                        )
        else:
            helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                    embeding, start_tokens=tf.zeros([batch], dtype=tf.int32) + word_dict['[START]'], 
                    end_token=word_dict['[END]'])

        decoder = tf.contrib.seq2seq.BasicDecoder(
                cell = attention_cell,
                helper = helper, 
                initial_state = initial_state)

        decoder_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(decoder, maximum_iterations=output_length)
        
        padding_need = padding_size - tf.shape(decoder_outputs.rnn_output)[1]
        padding_zeros = tf.zeros([batch, padding_need, vocab_size])
        decoder_rnn = tf.concat([decoder_outputs.rnn_output, padding_zeros], axis = 1)
    
    return decoder_rnn
    

In [17]:
def sequence_loss(prediction, label, decoder_length):
    
    one_hot_label =tf.one_hot(label, depth = tf.shape(prediction)[-1])
    per_example_loss = tf.nn.softmax_cross_entropy_with_logits_v2(labels = one_hot_label, logits = prediction)   
    per_example_loss = per_example_loss * decoder_length
    
#     per_example_loss = tf.where(decoder_length < 1, tf.stop_gradient(per_example_loss), per_example_loss)
    per_example_loss = tf.reduce_sum(per_example_loss, axis = -1) / tf.cast(tf.reduce_sum(decoder_length, axis = -1), dtype = tf.float32)
    
    return per_example_loss


In [18]:
import json
with open('data/vocab.json', 'r') as fp:
    word_dict = json.load(fp)
    
index2word = [word for word, i in word_dict.items()]

In [19]:
def inferQA(questions, index2word, word_dict, tokenizer, max_input_seq = 10):
    
    input_texts = tokenizer.tokenize(questions)
    input_ids = [word_dict[i] for i in input_texts if i in word_dict]
    
    if len(input_ids) > max_input_seq:
        raise 'sentence is too long'
    
    if len(input_ids) < max_input_seq:
        input_ids.extend([0] * (max_input_seq - len(input_ids)))
        
    input_ids = np.array(input_ids).reshape(-1, max_input_seq).astype(np.int64)
        
    print('question: %s' % ' '.join(questions))
    saver = tf.train.Saver()
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        full_path = tf.train.latest_checkpoint('model')
        if not full_path:
            raise 'no model file exist'
            
        saver.restore(sess, full_path)
        sess.run('test_init_op', feed_dict = {'input_plh:0': input_ids})
        prob= sess.run('prediction:0')[0]
        words = [index2word[i] for i in prob]
    print('answer: %s' % ' '.join(words))
        

In [20]:
class trainHook(tf.train.SessionRunHook):
    
    def __init__(self, metrics_op):
#       dict of {name: metrics}
        self.tensor_name = list(metrics_op.keys())
        self.tensor_eval = list(metrics_op.values())
        self.epoch = 0
        super().__init__()
        
    def before_run(self, run_context):
        return tf.train.SessionRunArgs(self.tensor_eval)
    
    
    def after_run(self, run_context, run_values):
        eval_result =  run_values.results
        self.epoch += 1
        if self.epoch % 100 == 0:
            for i, k in enumerate(eval_result): 
                eval_result, _ = k
                print('%s is %s' % (self.tensor_name[i], eval_result))

In [21]:
def build_input_fn(input_file, input_length, output_length, mode):

    feature_description = {
          "input_ids": tf.FixedLenFeature([input_length], tf.int64),
          "output_ids": tf.FixedLenFeature([output_length], tf.int64),
          }  
    def _parse_function(record):
        return tf.parse_single_example(record, feature_description)   

    def input_fn(params):
        
        batch_size = params['batch_size']
        epoch = params['epoch']
        
        if not mode == 'test':
            data = tf.data.TFRecordDataset(input_file)
            data = data.map(lambda x: _parse_function(x))

            if mode == 'train':
                data = data.repeat(epoch)
                data = data.shuffle(buffer_size=1000)
                
            data = data.batch(batch_size)
            iter_ = data.make_one_shot_iterator()
            features = iter_.get_next()
            return features["input_ids"], features["output_ids"]
        
        else:
            inputs = tf.placeholder(shape = [None, input_length], dtype = tf.int64, name = 'input_plh')
            data = tf.data.Dataset.from_tensor_slices(inputs)
            data = data.batch(batch_size = tf.shape(inputs)[0])
            return data
        
    return input_fn

In [32]:
def model_fn(features, labels, mode, params):
    
    word_dict = params['word_dict']
    attention_unit = params['attention_unit']
    vocab_size = len(word_dict)
    embeding_size = params['embeding_size']
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        train_output_ids = tf.zeros(shape = tf.shape(features))
    else:
        start_tokens = tf.zeros([tf.shape(labels)[0]], dtype=tf.int64) + word_dict['[START]']
        train_output_ids = tf.concat([tf.expand_dims(start_tokens, 1), labels], 1)
    
    with tf.variable_scope('embedding', reuse = tf.AUTO_REUSE):
        embeding = tf.get_variable(name = 'word_embedding', shape = [vocab_size, embeding_size], dtype = tf.float32,
                                      initializer=tf.contrib.layers.xavier_initializer())
    
    encoder_output, encoder_state, encoder_length = encoder(features, embeding, vocab_size, embeding_size)
    
    training = (mode == tf.estimator.ModeKeys.TRAIN)
    
    attention_define = (encoder_output, encoder_state, encoder_length, attention_unit) 
    
    decoder_output = decoder(train_output_ids, embeding, vocab_size, attention_define, padding_size = output_length, training = training)
    prediction = tf.argmax(decoder_output, axis = -1, name = 'prediction')
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        
        return tf.estimator.EstimatorSpec(mode, predictions=prediction)
    
    decoder_length_mask = tf.cast(labels > 0, dtype = tf.float32)
    
    per_example_loss = sequence_loss(decoder_output, labels, decoder_length_mask)
    loss = tf.metrics.mean(values=per_example_loss)
    total_loss = tf.reduce_mean(per_example_loss)   
    
    accuracy = tf.metrics.accuracy(prediction, labels, weights = decoder_length_mask)

    if mode == tf.estimator.ModeKeys.TRAIN:

        logging_hook = trainHook({'total_loss':loss})
        step = tf.train.get_global_step()
        train_op = tf.train.AdamOptimizer().minimize(total_loss, global_step=step)
        return tf.estimator.EstimatorSpec(mode, loss=total_loss, train_op=train_op, training_hooks = [logging_hook],
                                          eval_metric_ops={'accuracy':accuracy, 'loss':loss})

    else:     
        return tf.estimator.EstimatorSpec(mode, loss=total_loss, 
                                          eval_metric_ops={'accuracy_eval':accuracy, 'loss_eval':loss})
    

In [36]:
input_file_train = 'data/train'
input_file_eval = 'data/val'
input_length = 10
output_length = 10
input_fn_train = build_input_fn(input_file_train, input_length, output_length, mode = 'train')
input_fn_eval = build_input_fn(input_file_eval, input_length, output_length, mode = 'eval')

In [37]:
params = {'batch_size':32,
          'word_dict':word_dict,
          'attention_unit':32,
         'embeding_size':300,
         'epoch': 200}

estimator = tf.estimator.Estimator(model_fn, params = params, model_dir = 'model')

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f262ad885c0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [16]:
estimator.train(input_fn_train)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into model/model.ckpt.
INFO:tensorflow:loss = 9.610851, step = 0
total_loss is 6.221364
INFO:tensorflow:global_step/sec: 15.2753
INFO:tensorflow:loss = 5.630693, step = 100 (6.548 sec)
total_loss is 5.811168
INFO:tensorflow:global_step/sec: 18.0203
INFO:tensorflow:loss = 5.0350285, step = 200 (5.550 sec)
total_loss is 5.6153803
INFO:tensorflow:global_step/sec: 18.0608
INFO:tensorflow:loss = 4.6773825, step = 300 (5.537 sec)
total_loss is 5.4670386
INFO:tensorflow:global_step/sec: 18.1542
INFO:tensorflow:loss = 5.2585444, step = 400 (5.507 sec)
total_loss is 5.37291
INFO:tensorflow:global_step/sec: 18.1471
INFO:tensorflow:loss = 5.2963166, step = 500 (5.512 sec)
total_loss is 5.3030796
INFO:tensorflow:global_step/

INFO:tensorflow:loss = 3.446104, step = 6500 (5.566 sec)
total_loss is 4.123072
INFO:tensorflow:global_step/sec: 18.1701
INFO:tensorflow:loss = 3.4434063, step = 6600 (5.504 sec)
total_loss is 4.1134768
INFO:tensorflow:global_step/sec: 18.0447
INFO:tensorflow:loss = 3.772868, step = 6700 (5.541 sec)
total_loss is 4.1035767
INFO:tensorflow:global_step/sec: 18.0395
INFO:tensorflow:loss = 3.2680767, step = 6800 (5.543 sec)
total_loss is 4.093774
INFO:tensorflow:global_step/sec: 17.6985
INFO:tensorflow:loss = 3.5308895, step = 6900 (5.651 sec)
total_loss is 4.083849
INFO:tensorflow:global_step/sec: 18.0423
INFO:tensorflow:loss = 3.3465915, step = 7000 (5.542 sec)
total_loss is 4.0740833
INFO:tensorflow:global_step/sec: 17.9155
INFO:tensorflow:loss = 3.0736322, step = 7100 (5.582 sec)
total_loss is 4.063827
INFO:tensorflow:global_step/sec: 18.0475
INFO:tensorflow:loss = 3.176203, step = 7200 (5.541 sec)
total_loss is 4.054154
INFO:tensorflow:global_step/sec: 18.1432
INFO:tensorflow:loss = 3

total_loss is 3.5386734
INFO:tensorflow:global_step/sec: 17.9859
INFO:tensorflow:loss = 3.1899035, step = 13200 (5.559 sec)
total_loss is 3.5309072
INFO:tensorflow:global_step/sec: 17.9642
INFO:tensorflow:loss = 2.59535, step = 13300 (5.566 sec)
total_loss is 3.5231984
INFO:tensorflow:global_step/sec: 18.0101
INFO:tensorflow:loss = 2.7223918, step = 13400 (5.553 sec)
total_loss is 3.5157106
INFO:tensorflow:global_step/sec: 18.1224
INFO:tensorflow:loss = 2.2371633, step = 13500 (5.518 sec)
total_loss is 3.5080245
INFO:tensorflow:global_step/sec: 17.9073
INFO:tensorflow:loss = 2.4818575, step = 13600 (5.584 sec)
total_loss is 3.5004303
INFO:tensorflow:global_step/sec: 17.9995
INFO:tensorflow:loss = 2.1313171, step = 13700 (5.556 sec)
total_loss is 3.4926772
INFO:tensorflow:global_step/sec: 18.1714
INFO:tensorflow:loss = 2.24816, step = 13800 (5.503 sec)
total_loss is 3.4847803
INFO:tensorflow:global_step/sec: 18.0591
INFO:tensorflow:loss = 2.5403225, step = 13900 (5.537 sec)
total_loss i

INFO:tensorflow:loss = 1.8819019, step = 19800 (5.605 sec)
total_loss is 3.0761673
INFO:tensorflow:global_step/sec: 17.9023
INFO:tensorflow:loss = 1.8070006, step = 19900 (5.586 sec)
total_loss is 3.0701234
INFO:tensorflow:global_step/sec: 17.8284
INFO:tensorflow:loss = 1.76174, step = 20000 (5.608 sec)
total_loss is 3.0639675
INFO:tensorflow:global_step/sec: 17.9104
INFO:tensorflow:loss = 1.6657674, step = 20100 (5.584 sec)
total_loss is 3.057919
INFO:tensorflow:global_step/sec: 18.0264
INFO:tensorflow:loss = 1.5398386, step = 20200 (5.547 sec)
total_loss is 3.0519633
INFO:tensorflow:global_step/sec: 17.9487
INFO:tensorflow:loss = 1.8754101, step = 20300 (5.572 sec)
total_loss is 3.0461683
INFO:tensorflow:global_step/sec: 17.9774
INFO:tensorflow:loss = 1.7915928, step = 20400 (5.562 sec)
total_loss is 3.0402534
INFO:tensorflow:global_step/sec: 18.0823
INFO:tensorflow:loss = 1.744911, step = 20500 (5.531 sec)
total_loss is 3.0343237
INFO:tensorflow:global_step/sec: 17.9248
INFO:tensorf

INFO:tensorflow:loss = 1.3445169, step = 26400 (5.559 sec)
total_loss is 2.7247744
INFO:tensorflow:global_step/sec: 18.0604
INFO:tensorflow:loss = 1.4772356, step = 26500 (5.538 sec)
total_loss is 2.7201142
INFO:tensorflow:global_step/sec: 17.9606
INFO:tensorflow:loss = 1.3917872, step = 26600 (5.567 sec)
total_loss is 2.7154148
INFO:tensorflow:global_step/sec: 18.1832
INFO:tensorflow:loss = 1.4028897, step = 26700 (5.500 sec)
total_loss is 2.710753
INFO:tensorflow:global_step/sec: 17.9768
INFO:tensorflow:loss = 1.3969895, step = 26800 (5.563 sec)
total_loss is 2.7062182
INFO:tensorflow:global_step/sec: 18.0012
INFO:tensorflow:loss = 1.3994371, step = 26900 (5.554 sec)
total_loss is 2.7014306
INFO:tensorflow:global_step/sec: 18.2021
INFO:tensorflow:loss = 1.6541698, step = 27000 (5.494 sec)
total_loss is 2.6969216
INFO:tensorflow:global_step/sec: 18.0728
INFO:tensorflow:loss = 1.1702385, step = 27100 (5.533 sec)
total_loss is 2.6923983
INFO:tensorflow:global_step/sec: 18.0015
INFO:tens

...

total_loss is 1.298807
INFO:tensorflow:global_step/sec: 18.0729
INFO:tensorflow:loss = 0.34063476, step = 125000 (5.532 sec)
total_loss is 1.2983623
INFO:tensorflow:global_step/sec: 18.146
INFO:tensorflow:loss = 0.9094278, step = 125100 (5.511 sec)
total_loss is 1.2979234
INFO:tensorflow:global_step/sec: 18.1573
INFO:tensorflow:loss = 0.6985357, step = 125200 (5.508 sec)
total_loss is 1.2974819
INFO:tensorflow:global_step/sec: 17.925
INFO:tensorflow:loss = 0.74543977, step = 125300 (5.578 sec)
total_loss is 1.2970672
INFO:tensorflow:global_step/sec: 18.0734
INFO:tensorflow:loss = 0.66709834, step = 125400 (5.533 sec)
total_loss is 1.2966757
INFO:tensorflow:global_step/sec: 18.2799
INFO:tensorflow:loss = 0.79078436, step = 125500 (5.471 sec)
total_loss is 1.296227
INFO:tensorflow:global_step/sec: 18.2408
INFO:tensorflow:loss = 0.82052773, step = 125600 (5.482 sec)
total_loss is 1.2957982
INFO:tensorflow:global_step/sec: 18.3635
INFO:tensorflow:loss = 0.7169689, step = 125700 (5.446 sec)

total_loss is 1.2720109
INFO:tensorflow:global_step/sec: 18.1787
INFO:tensorflow:loss = 0.7675737, step = 131500 (5.501 sec)
total_loss is 1.2716235
INFO:tensorflow:global_step/sec: 18.1726
INFO:tensorflow:loss = 0.85699546, step = 131600 (5.503 sec)
total_loss is 1.2712171
INFO:tensorflow:global_step/sec: 18.0134
INFO:tensorflow:loss = 0.99723244, step = 131700 (5.552 sec)
total_loss is 1.2708261
INFO:tensorflow:global_step/sec: 18.1768
INFO:tensorflow:loss = 0.71362877, step = 131800 (5.501 sec)
total_loss is 1.2704363
INFO:tensorflow:global_step/sec: 18.1468
INFO:tensorflow:loss = 0.63456756, step = 131900 (5.511 sec)
total_loss is 1.2700263
INFO:tensorflow:global_step/sec: 18.1004
INFO:tensorflow:loss = 0.8546218, step = 132000 (5.528 sec)
total_loss is 1.2696309
INFO:tensorflow:global_step/sec: 18.0582
INFO:tensorflow:loss = 0.6393244, step = 132100 (5.535 sec)
total_loss is 1.2692683
INFO:tensorflow:global_step/sec: 18.0694
INFO:tensorflow:loss = 0.59221923, step = 132200 (5.533 

INFO:tensorflow:loss = 0.6463091, step = 138000 (5.550 sec)
total_loss is 1.2471794
INFO:tensorflow:global_step/sec: 18.3915
INFO:tensorflow:loss = 0.7120867, step = 138100 (5.438 sec)
total_loss is 1.2468055
INFO:tensorflow:global_step/sec: 18.1472
INFO:tensorflow:loss = 0.964744, step = 138200 (5.510 sec)
total_loss is 1.2464641
INFO:tensorflow:global_step/sec: 18.1477
INFO:tensorflow:loss = 0.64567125, step = 138300 (5.510 sec)
total_loss is 1.2461152
INFO:tensorflow:global_step/sec: 18.1957
INFO:tensorflow:loss = 0.73395514, step = 138400 (5.496 sec)
total_loss is 1.2457631
INFO:tensorflow:global_step/sec: 18.3732
INFO:tensorflow:loss = 0.6366248, step = 138500 (5.443 sec)
total_loss is 1.2454017
INFO:tensorflow:global_step/sec: 18.0939
INFO:tensorflow:loss = 0.8446541, step = 138600 (5.526 sec)
total_loss is 1.2450407
INFO:tensorflow:global_step/sec: 18.1092
INFO:tensorflow:loss = 0.85912156, step = 138700 (5.522 sec)
total_loss is 1.2446904
INFO:tensorflow:global_step/sec: 18.441

INFO:tensorflow:loss = 1.7541653, step = 144500 (5.430 sec)
total_loss is 1.2247107
INFO:tensorflow:global_step/sec: 18.028
INFO:tensorflow:loss = 0.85983074, step = 144600 (5.547 sec)
total_loss is 1.2243907
INFO:tensorflow:global_step/sec: 18.0057
INFO:tensorflow:loss = 1.0048505, step = 144700 (5.555 sec)
total_loss is 1.2240553
INFO:tensorflow:global_step/sec: 18.006
INFO:tensorflow:loss = 0.7744864, step = 144800 (5.553 sec)
total_loss is 1.2237324
INFO:tensorflow:global_step/sec: 18.0695
INFO:tensorflow:loss = 0.986666, step = 144900 (5.535 sec)
total_loss is 1.2234001
INFO:tensorflow:global_step/sec: 17.9148
INFO:tensorflow:loss = 0.54192454, step = 145000 (5.581 sec)
total_loss is 1.2230713
INFO:tensorflow:global_step/sec: 18.0402
INFO:tensorflow:loss = 0.7640409, step = 145100 (5.544 sec)
total_loss is 1.2227738
INFO:tensorflow:global_step/sec: 18.2911
INFO:tensorflow:loss = 0.9681878, step = 145200 (5.466 sec)
total_loss is 1.2224687
INFO:tensorflow:global_step/sec: 18.2304
I

INFO:tensorflow:loss = 0.6781868, step = 151100 (5.541 sec)
total_loss is 1.2041256
INFO:tensorflow:global_step/sec: 18.1707
INFO:tensorflow:loss = 0.7037245, step = 151200 (5.508 sec)
total_loss is 1.203867
INFO:tensorflow:global_step/sec: 18.0914
INFO:tensorflow:loss = 0.7839987, step = 151300 (5.523 sec)
total_loss is 1.2035866
INFO:tensorflow:global_step/sec: 18.2911
INFO:tensorflow:loss = 0.9056811, step = 151400 (5.467 sec)
total_loss is 1.2032853
INFO:tensorflow:global_step/sec: 18.0782
INFO:tensorflow:loss = 0.7631686, step = 151500 (5.533 sec)
total_loss is 1.202987
INFO:tensorflow:global_step/sec: 18.2984
INFO:tensorflow:loss = 0.8193743, step = 151600 (5.464 sec)
total_loss is 1.2027026
INFO:tensorflow:global_step/sec: 17.9441
INFO:tensorflow:loss = 0.6439753, step = 151700 (5.573 sec)
total_loss is 1.2023805
INFO:tensorflow:global_step/sec: 18.2213
INFO:tensorflow:loss = 0.6877843, step = 151800 (5.488 sec)
INFO:tensorflow:Saving checkpoints for 151821 into model/model.ckpt

In [38]:
estimator.evaluate(input_fn_eval)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-05-07T06:30:50Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from model/model.ckpt-152225
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-05-07-06:30:59
INFO:tensorflow:Saving dict for global step 152225: accuracy_eval = 0.08100116, global_step = 152225, loss = 19.955862, loss_eval = 19.948042
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 152225: model/model.ckpt-152225


{'accuracy_eval': 0.08100116,
 'loss': 19.955862,
 'loss_eval': 19.948042,
 'global_step': 152225}